In [101]:
import tensorflow as tf 
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd 
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler

In [102]:
xy = pd.read_csv('무Model.csv')


In [103]:
%matplotlib notebook
plt.plot(xy['무가격']) # 전체 종가
xy.index = xy['날짜']
xy = xy.drop(['날짜'],axis=1)

<IPython.core.display.Javascript object>

In [136]:
seqLength = 21 # window size 
dataDim = 27 # 시가, 고가, 저가, 거래량 , 종가
hiddenDim = 10 
outputDim = 1
lr = 0.01
iterations = 500

trainSize = int(len(xy)*0.7)
trainSet = xy[0:trainSize]
testSet = xy[trainSize-seqLength:]

scaler = MinMaxScaler()

trainSet = scaler.fit_transform(trainSet)
testSet = scaler.fit_transform(testSet)

# trainSet = MinMax(trainSet)
# testSet = MinMax(testSet)

In [138]:

# 7일간의 5가지 데이터(시가, 종가, 고가, 저가, 거래량)를 받아와서 
# 바로 다음 날의 종가를 예측하는 모델로 구성

def buildDataSet(timeSeries, seqLength):
    xdata = []
    ydata = [] 
    a = len(timeSeries)
    for i in range(0, a-seqLength):
        tx = timeSeries[i:i+seqLength,:-1]
        ty = timeSeries[i+seqLength,[-1]]
        xdata.append(tx)
        ydata.append(ty)
    return np.array(xdata), np.array(ydata)
    
trainX, trainY=buildDataSet(trainSet, seqLength)
testX, testY=buildDataSet(testSet, seqLength)

In [139]:
# 모델 구성 

# First, let's define a RNN Cell, as a layer subclass.
from tensorflow import keras
from tensorflow.keras import layers

model = keras.Sequential()

model.add(layers.SimpleRNN(units=10, activation='tanh',input_shape=[21,26]))

model.add(layers.Dense(1))

model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_8 (SimpleRNN)    (None, 10)                370       
                                                                 
 dense_8 (Dense)             (None, 1)                 11        
                                                                 
Total params: 381
Trainable params: 381
Non-trainable params: 0
_________________________________________________________________


In [140]:
# 모델 학습과정 설정 
model.compile(loss='mse', optimizer='adam', metrics=['mae'])

# 모델 트레이닝 
hist = model.fit(trainX, trainY, epochs=1000, batch_size=16)

Train on 1497 samples
Epoch 1/1000
1497/1497 [==============================] - 0s 177us/sample - loss: 0.0356 - mean_absolute_error: 0.1443
Epoch 2/1000
1497/1497 [==============================] - 0s 114us/sample - loss: 0.0137 - mean_absolute_error: 0.0895
Epoch 3/1000
1497/1497 [==============================] - 0s 156us/sample - loss: 0.0107 - mean_absolute_error: 0.0788
Epoch 4/1000
1497/1497 [==============================] - 0s 125us/sample - loss: 0.0095 - mean_absolute_error: 0.0738
Epoch 5/1000
1497/1497 [==============================] - 0s 133us/sample - loss: 0.0088 - mean_absolute_error: 0.0705
Epoch 6/1000
1497/1497 [==============================] - 0s 123us/sample - loss: 0.0085 - mean_absolute_error: 0.0694
Epoch 7/1000
1497/1497 [==============================] - 0s 119us/sample - loss: 0.0080 - mean_absolute_error: 0.0670
Epoch 8/1000
1497/1497 [==============================] - 0s 119us/sample - loss: 0.0079 - mean_absolute_error: 0.0668
Epoch 9/1000
1497/1497 [==

In [141]:
res = model.evaluate(testX, testY, batch_size=16)
print("loss",res[0],"mae",res[1])

loss 0.0770260909437767 mae 0.20343772


C:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\engine\training_v1.py:2057: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


In [142]:
#7 모델 사용
xhat = testX
yhat = model.predict(xhat)
print(testY)
print(yhat)

print("Evaluate : {}".format(np.average((yhat - testY)**2)))

[[0.09885827]
 [0.15379668]
 [0.1459063 ]
 [0.1713591 ]
 [0.12214512]
 [0.09805066]
 [0.09461018]
 [0.10974825]
 [0.15050241]
 [0.19486263]
 [0.18544925]
 [0.16457801]
 [0.12938905]
 [0.09284023]
 [0.10504131]
 [0.10667513]
 [0.10814686]
 [0.08981327]
 [0.09263295]
 [0.0848046 ]
 [0.09833324]
 [0.0996263 ]
 [0.10300927]
 [0.09667301]
 [0.07360913]
 [0.09345421]
 [0.1308497 ]
 [0.14606074]
 [0.13018111]
 [0.16095662]
 [0.15696056]
 [0.10890005]
 [0.0934705 ]
 [0.06581668]
 [0.05562965]
 [0.04795456]
 [0.03889333]
 [0.05863073]
 [0.06877429]
 [0.06317513]
 [0.06172352]
 [0.06594353]
 [0.07744133]
 [0.06579317]
 [0.08907194]
 [0.09643996]
 [0.11732885]
 [0.11622666]
 [0.11221918]
 [0.04059655]
 [0.09578648]
 [0.07282326]
 [0.09766695]
 [0.09159495]
 [0.1220099 ]
 [0.12089479]
 [0.0632318 ]
 [0.13405671]
 [0.14654404]
 [0.0920989 ]
 [0.09057588]
 [0.1073881 ]
 [0.14127294]
 [0.16530967]
 [0.14557548]
 [0.13055714]
 [0.11813391]
 [0.10317579]
 [0.10936183]
 [0.11517527]
 [0.1169875 ]
 [0.10

C:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\engine\training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


In [143]:
#원래 값으로 되돌리기 

xy2 = xy.to_numpy()

# predict1 = back_MinMax(xy2[trainSize-seqLength:,[-1]],yhat)
# actual = back_MinMax(xy2[trainSize-seqLength:,[-1]],testY)
# print("예측값",predict1)
# print("실제값",actual)

print(yhat.shape)
print(testY.shape)

(651, 1)
(651, 1)


In [144]:
%matplotlib notebook
plt.figure(figsize=(20,10))
plt.plot(yhat[:30], label = "Mu_RNN")
plt.plot(testY[:30],label = "Real")

plt.legend(prop={'size': 20})
plt.savefig('결과/무모델.png')


<IPython.core.display.Javascript object>